In [1]:
import torch 
import torch.nn as nn

from torch.autograd import Variable
import numpy as np


# Hyper Parameters

input_size = 13
hidden_size = 60
num_layers = 2
num_classes = 2
batch_size = 1
num_epochs = 2
learning_rate = 0.01

In [2]:
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav



In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        print (out.size())
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

In [5]:
# create a many to one LSTM , just take last output timestep as output 
rnn = RNN(input_size, hidden_size, num_layers, num_classes)

In [6]:
# Loss and Optimizer
#criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

'''
>>> input = autograd.Variable(torch.randn(3, 5), requires_grad=True)
>>> target = autograd.Variable(torch.LongTensor(3).random_(5))
>>> output = loss(input, target)
>>> output.backward()
'''
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [8]:
mfcc1 = audio_to_mfcc('/home/saurabh/Documents/audio_classification/data/lizzie.wav')
#print(mfcc1.shape)
mfcc2 = audio_to_mfcc('/home/saurabh/Documents/audio_classification/data/boy.wav')
#print(mfcc2.shape)



temp =  mfcc1[ : , np.newaxis , :]
temp2=  mfcc2[ : , np.newaxis , :]
#print(temp.shape)
input_var = Variable(torch.Tensor(temp))


for epoch in range(2):
    outputs = rnn(input_var)
#print(outputs[999])
    final_output = outputs[999]

    final_output_numpy=final_output.data.numpy()[np.newaxis,:]
    final_output = torch.from_numpy(final_output_numpy)
#print(final_output_numpy.shape)

#print(final_output.size())

    label = Variable(torch.LongTensor([1]))
#print (label.size())
    loss = criterion(Variable(final_output, requires_grad=True), label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)
    
    


torch.Size([1000, 1, 60])
Variable containing:
 0.6407
[torch.FloatTensor of size 1]

torch.Size([1000, 1, 60])
Variable containing:
 0.6407
[torch.FloatTensor of size 1]



In [ ]:
# for i = 1 to number of examples 
#convert each example to mfcc array , and pass to LSTM

for epoch in range(num_epochs):
    

